In [1]:
!python --version

Python 3.7.6


In [2]:
import numpy as np
import pandas as pd
import seaborn as sys
import matplotlib.pyplot as plt
from IPython.display import display_html 

In [3]:
df1 = pd.read_csv("groceries.csv", header=None)
df2 = pd.read_csv("sup_sales.csv")

In [4]:
df1 = df1.fillna(' ')
df1[3:15]

,0,1,2,3,4,5,6,7,8,9
3,pip fruit,yogurt,cream cheese,meat spreads,,,,,,
4,other vegetables,whole milk,condensed milk,long life bakery product,,,,,,
5,whole milk,butter,yogurt,rice,abrasive cleaner,,,,,
6,rolls/buns,,,,,,,,,
7,other vegetables,UHT-milk,rolls/buns,bottled beer,liquor (appetizer),,,,,
8,potted plants,,,,,,,,,
9,whole milk,cereals,,,,,,,,
10,tropical fruit,other vegetables,white bread,bottled water,chocolate,,,,,
11,citrus fruit,tropical fruit,whole milk,butter,curd,yogurt,flour,bottled water,dishes,
12,beef,,,,,,,,,


In [5]:
df2.head(10)

,Customer type,Gender,Product line,Quantity,Payment,Rating
0,Member,Female,Health and beauty,7,Ewallet,9.1
1,Normal,Female,Electronic accessories,5,Cash,9.6
2,Normal,Male,Home and lifestyle,7,Credit card,7.4
3,Member,Male,Health and beauty,8,Ewallet,8.4
4,Normal,Male,Sports and travel,7,Ewallet,5.3
5,Normal,Male,Electronic accessories,7,Ewallet,4.1
6,Member,Female,Electronic accessories,6,Ewallet,5.8
7,Normal,Female,Home and lifestyle,10,Ewallet,8.0
8,Member,Female,Health and beauty,2,Credit card,7.2
9,Member,Female,Food and beverages,3,Credit card,5.9


In [6]:
transactions = []
for i in range(len(df1)):
    transactions.append([str(df1.values[i,j]) for j in range(0, 10)])

## Apriori

In [7]:
# Inspecting elements
transactions[:3]

[['citrus fruit',
  'semi-finished bread',
  'margarine',
  'ready soups',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' '],
 ['tropical fruit', 'yogurt', 'coffee', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 ['whole milk', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']]

In [8]:
# Training Apriori on the dataset
# The hyperparameters choosen on this training are:
# min_support = items bought more than 3 times a day * 7 days (week) / 7500 customers = 0.0028
# min_confidence: at least 20%, min_lift = minimum of 3 (less than that is too low)
# min_length: we want at least 2 items to be associated. No point in having a single item in the result

from apyori import apriori
rules = apriori(transactions, min_support = 0.003, min_confidence = 0.2, min_lift = 3, min_length = 2)

In [9]:
# Visualising the results
results = list(rules)

In [10]:
lift = []
association = []
for i in range (0, len(results)):
    lift.append(results[:len(results)][i][2][0][3])
    association.append(list(results[:len(results)][i][0]))

In [11]:
rank = pd.DataFrame([association, lift]).transpose()
rank.columns = ['Association', 'Lift']

In [12]:
# Show top 10 higher lift scores
results = rank.sort_values('Lift', ascending=False)
results.head(10)

,Association,Lift
11,"[flour, , sugar]",8.60243
4,"[flour, sugar]",8.60243
118,"[other vegetables, , root vegetables, tropica...",6.76343
94,"[root vegetables, tropical fruit, other vegeta...",6.76343
99,"[yogurt, whole milk, tropical fruit, curd]",6.07591
123,"[yogurt, tropical fruit, curd, whole milk, ]",6.07591
2,"[bottled beer, liquor]",5.94348
9,"[bottled beer, liquor, ]",5.94348
125,"[other vegetables, pip fruit, root vegetables,...",5.86038
101,"[pip fruit, root vegetables, other vegetables,...",5.86038


In [14]:
#results.plot(x=results['Association'] , y=results['Lift'])

## ECLAT

In [15]:
# Putting all transactions in a single list
itens = []
for i in range(0, len(transactions)):
    itens.extend(transactions[i])

# Finding unique items from transactions and removing nan
uniqueItems = list(set(itens))
uniqueItems.remove('nan')

ValueError: list.remove(x): x not in list

In [16]:
uniqueItems

['beverages',
 'baking powder',
 'frozen vegetables',
 'brown bread',
 'yogurt',
 'male cosmetics',
 'berries',
 'cream',
 'liver loaf',
 'baby cosmetics',
 'whipped/sour cream',
 'tropical fruit',
 'zwieback',
 'dish cleaner',
 'cling film/bags',
 'cookware',
 'sweet spreads',
 'rolls/buns',
 'herbs',
 'oil',
 'specialty cheese',
 'frozen meals',
 'meat',
 'canned vegetables',
 'cereals',
 'dessert',
 'specialty bar',
 'roll products',
 'dog food',
 'other vegetables',
 'tea',
 'kitchen towels',
 'mayonnaise',
 'ready soups',
 'liquor (appetizer)',
 'dishes',
 'semi-finished bread',
 'napkins',
 'pickled vegetables',
 'pasta',
 'hair spray',
 'curd',
 'red/blush wine',
 'finished products',
 'shopping bags',
 'flower soil/fertilizer',
 'tidbits',
 'frozen dessert',
 'ham',
 'spread cheese',
 'liquor',
 'potato products',
 'long life bakery product',
 'waffles',
 'coffee',
 'fish',
 'ketchup',
 'frozen fish',
 'jam',
 'packaged fruit/vegetables',
 'artif. sweetener',
 'abrasive cleaner

In [17]:
pair = []
for j in range(0, len(uniqueItems)):
    k = 1;
    while k <= len(uniqueItems):
        try:
            pair.append([uniqueItems[j], uniqueItems[j+k]])
        except IndexError:
            pass
        k = k + 1;       

In [18]:
pair

[['beverages', 'baking powder'],
 ['beverages', 'frozen vegetables'],
 ['beverages', 'brown bread'],
 ['beverages', 'yogurt'],
 ['beverages', 'male cosmetics'],
 ['beverages', 'berries'],
 ['beverages', 'cream'],
 ['beverages', 'liver loaf'],
 ['beverages', 'baby cosmetics'],
 ['beverages', 'whipped/sour cream'],
 ['beverages', 'tropical fruit'],
 ['beverages', 'zwieback'],
 ['beverages', 'dish cleaner'],
 ['beverages', 'cling film/bags'],
 ['beverages', 'cookware'],
 ['beverages', 'sweet spreads'],
 ['beverages', 'rolls/buns'],
 ['beverages', 'herbs'],
 ['beverages', 'oil'],
 ['beverages', 'specialty cheese'],
 ['beverages', 'frozen meals'],
 ['beverages', 'meat'],
 ['beverages', 'canned vegetables'],
 ['beverages', 'cereals'],
 ['beverages', 'dessert'],
 ['beverages', 'specialty bar'],
 ['beverages', 'roll products'],
 ['beverages', 'dog food'],
 ['beverages', 'other vegetables'],
 ['beverages', 'tea'],
 ['beverages', 'kitchen towels'],
 ['beverages', 'mayonnaise'],
 ['beverages', 'r

In [19]:
%%time
score = []
for i in pair:
    cond = []
    for item in i:
        cond.append('("%s") in s' %item)
    mycode = ('[s for s in transactions if ' + ' and '.join(cond) + ']')
    #mycode = "print 'hello world'"
    score.append(len(eval(mycode))/1000.)

Wall time: 30.2 s


In [20]:
ranking_ECLAT = pd.DataFrame([pair, score]).transpose()
ranking_ECLAT.columns = ['Pair', 'Score']

In [21]:
ranking_ECLAT.sort_values('Score', ascending=False).head(10)

,Pair,Score
8552,"[ , whole milk]",2.509
4498,"[other vegetables, ]",1.901
2764,"[rolls/buns, ]",1.754
8535,"[ , soda]",1.575
723,"[yogurt, ]",1.351
8581,"[ , root vegetables]",1.071
1843,"[tropical fruit, ]",1.031
8566,"[ , bottled water]",0.99
8577,"[ , sausage]",0.923
8619,"[ , citrus fruit]",0.813


In [22]:
# Creating trios
trio = []
for j in range(0, len(uniqueItems)):
    for k in range(j, len(uniqueItems)):
        for l in range(k, len(uniqueItems)):
            if (k != j) and (j != l) and (k != l):
                try:
                    trio.append([uniqueItems[j], uniqueItems[j+k], uniqueItems[j+l]])
                except IndexError:
                    pass 

In [23]:
%%time
score_trio = []
for i in trio:
    cond = []
    for item in i:
        cond.append('("%s") in s' %item)
    mycode = ('[s for s in transactions if ' + ' and '.join(cond) + ']')
    #mycode = "print 'hello world'"
    score_trio.append(len(eval(mycode))/1000.)

Wall time: 12min 53s


In [24]:
ranking_ECLAT_trio = pd.DataFrame([trio, score_trio]).transpose()
ranking_ECLAT_trio.columns = ['Trio', 'Score']
ranking_ECLAT_trio.sort_values('Score', ascending=False).head(10)

,Trio,Score
286581,"[other vegetables, , whole milk]",0.732
61206,"[yogurt, , whole milk]",0.534
199175,"[rolls/buns, , whole milk]",0.515
286610,"[other vegetables, , root vegetables]",0.464
141594,"[tropical fruit, , whole milk]",0.412
57152,"[yogurt, other vegetables, ]",0.409
137540,"[tropical fruit, other vegetables, ]",0.351
199158,"[rolls/buns, , soda]",0.329
55418,"[yogurt, rolls/buns, ]",0.303
131009,"[whipped/sour cream, , whole milk]",0.299


In [25]:
import pyfpgrowth

In [26]:
patterns = pyfpgrowth. find_frequent_patterns(transactions,50)

In [27]:
rules = pyfpgrowth. generate_association_rules(patterns,0.8)

In [28]:
def support_count(rhs):
    count =0 
    rhs = set(rhs)
    for j in df1['']

SyntaxError: invalid syntax (<ipython-input-28-b9114d9d5b13>, line 4)